In [6]:
!git clone https://github.com/css20090922/address_extraction.git

Cloning into 'address_extraction'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 15 (delta 2), reused 9 (delta 0), pack-reused 0
Unpacking objects: 100% (15/15), done.


In [7]:
%cd address_extraction/

/content/address_extraction


In [8]:
!pip install transformers

In [9]:
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
import pandas as pd
import csv
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig,TFBertForTokenClassification,create_optimizer
from sklearn.model_selection import train_test_split

In [10]:
MAX_LEN = 128
configuration = BertConfig()

In [11]:
MODEL_NAME = "bert-base-cased"
slow_tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
save_path = "bert_base_cased/"
if not os.path.exists(save_path):
    os.makedirs(save_path)
slow_tokenizer.save_pretrained(save_path)

tokenizer = BertWordPieceTokenizer("bert_base_cased/vocab.txt", lowercase=True)

In [23]:
df = pd.read_csv('/content/address_extraction/datasets/train.csv', encoding='latin-1')
df.set_index('id', inplace=True)

df['POI'] = df['POI/street'].str.split("/",n=2,expand=True)[0]
df['street'] = df['POI/street'].str.split("/",n=2,expand=True)[1]
# x_train,x_test = train_test_split(df,test_size=0.20,shuffle=False)

df.head()

,raw_address,POI/street,POI,street
id,,,,
0,jl kapuk timur delta sili iii lippo cika 11 a ...,/jl kapuk timur delta sili iii lippo cika,,jl kapuk timur delta sili iii lippo cika
1,"aye, jati sampurna",/,,
2,setu siung 119 rt 5 1 13880 cipayung,/siung,,siung
3,"toko dita, kertosono",toko dita/,toko dita,
4,jl. orde baru,/jl. orde baru,,jl. orde baru


記得tokenize前要先個別取出來

In [29]:
class create_dataset:
  def __init__(self, raw_address, street, POI):
        self.raw_address = raw_address
        self.POI = POI
        self.street = street
        self.skip = False
  def preprocess(self):
    raw_address = self.raw_address
    POI = self.POI
    street = self.street
    
    # Tokenize tokenized_raw_address
    tokenized_raw_address = tokenizer.encode(raw_address)
    tokenized_POI = tokenizer.encode(POI)
    tokenized_street = tokenizer.encode(street)

    input_ids = tokenized_raw_address
    
    attention_mask = [1] * len(input_ids)

    padding_length = max_len - len(input_ids)

    if padding_length > 0:  # pad
        input_ids = input_ids + ([0] * padding_length)
        attention_mask = attention_mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)
    elif padding_length < 0:  # skip
        self.skip = True
        return
    self.input_ids = input_ids
    self.attention_mask = attention_mask